In [3]:
import sys
import os
# Agregar el directorio '' al sys.path
sys.path.append(os.path.abspath('../'))


In [4]:
from Function import cargar_dataset
from  limpiar_customer_id import limpiar_customer_id
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from openpyxl import load_workbook 
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam


# Creating a Predictive Model Using Random Forest

In [5]:
file_path = '/Users/Usuario/Desktop/prueba_citec/data_set/OnlineRetail.xlsx'
# Cargar el dataset
df = cargar_dataset(file_path)

In [27]:
# Prepare the data
limpiar_customer_id(df)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Dia,MesN,Mes,Año,ultima_compra,dias_desde_ultima_compra,compra_30_dias
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,Wednesday,12,December,2010,2011-02-10 14:38:00,301,0
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,Wednesday,12,December,2010,2011-02-10 14:38:00,301,0
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,Wednesday,12,December,2010,2011-02-10 14:38:00,301,0
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,Wednesday,12,December,2010,2011-02-10 14:38:00,301,0
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,Wednesday,12,December,2010,2011-02-10 14:38:00,301,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680,France,Friday,12,December,2011,2011-12-09 12:50:00,0,1
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680,France,Friday,12,December,2011,2011-12-09 12:50:00,0,1
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680,France,Friday,12,December,2011,2011-12-09 12:50:00,0,1
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680,France,Friday,12,December,2011,2011-12-09 12:50:00,0,1


In [8]:
# Crear una columna con la fecha de la última compra
df['ultima_compra'] = df.groupby('CustomerID')['InvoiceDate'].transform('max')

In [10]:
# Calcular los días desde la última compra
df['dias_desde_ultima_compra'] = (df['InvoiceDate'].max() - df['ultima_compra']).dt.days

In [11]:
# Crear la variable objetivo: 1 si el cliente compró en los últimos 30 días, 0 si no
df['compra_30_dias'] = np.where(df['dias_desde_ultima_compra'] <= 30, 1, 0)

In [12]:
# Calcular los días desde la última compra
df['dias_desde_ultima_compra'] = (df['InvoiceDate'].max() - df['ultima_compra']).dt.days

In [13]:
# Crear la variable objetivo: 1 si el cliente compró en los últimos 30 días, 0 si no
df['compra_30_dias'] = np.where(df['dias_desde_ultima_compra'] <= 30, 1, 0)


In [14]:
# Seleccionar las características y la variable objetivo
X = df[['dias_desde_ultima_compra']]
y = df['compra_30_dias']



 # Split data into training and test sets


In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train the model

In [16]:
# Crear el modelo
modelo = RandomForestClassifier(n_estimators=100, random_state=42)

# Entrenar el modelo
modelo.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

# Evaluate the model

In [17]:
# Hacer predicciones
y_pred = modelo.predict(X_test)

# Evaluar el modelo
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 1.0
Confusion Matrix:
 [[ 37940      0]
 [     0 124633]]
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     37940
           1       1.00      1.00      1.00    124633

    accuracy                           1.00    162573
   macro avg       1.00      1.00      1.00    162573
weighted avg       1.00      1.00      1.00    162573



#Predict probability

In [18]:
# Predecir la probabilidad de una nueva compra para un nuevo cliente
nuevos_datos = pd.DataFrame({'dias_desde_ultima_compra': [10, 20, 40, 100,1]})
probabilidades = modelo.predict_proba(nuevos_datos)[:, 1]
print("Probabilidades de nueva compra en los próximos 30 días:", probabilidades)

Probabilidades de nueva compra en los próximos 30 días: [1. 1. 0. 0. 1.]


# Creating a Predictive Model Using 


In [19]:
# Crear el modelo de regresión logística
modelo = LogisticRegression()
# Entrenar el modelo
modelo.fit(X_train, y_train)


LogisticRegression()

In [20]:
# Hacer predicciones
y_pred = modelo.predict(X_test)

# Evaluar el modelo
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


Accuracy: 1.0
Confusion Matrix:
 [[ 37940      0]
 [     0 124633]]
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     37940
           1       1.00      1.00      1.00    124633

    accuracy                           1.00    162573
   macro avg       1.00      1.00      1.00    162573
weighted avg       1.00      1.00      1.00    162573



In [25]:
# Predecir la probabilidad de una nueva compra para un nuevo cliente
nuevos_datos = pd.DataFrame({'dias_desde_ultima_compra': [10, 20, 160]})
probabilidades = modelo.predict_proba(nuevos_datos)[:, 1]
print("Probabilidades de nueva compra en los próximos 30 días:", probabilidades)

Probabilidades de nueva compra en los próximos 30 días: [1. 1. 0.]
